****Create a list of 5 minute window tuples that contain the start and finishing time. This will be used to query the database in moving windows.****

In [9]:
import datetime

times = []
a = datetime.datetime(100,1,1,0,0,0)
for i in range(0, 288):
    b = a + datetime.timedelta(minutes = 5)
    time = (str((a + datetime.timedelta(seconds=1)).time())), str(b.time())
    if i == 0:
        time = (str(a.time()), str(b.time()))
    a = b
    times.append(time)
times[len(times)-1] = ('23:55:01', '23:59:59')

****Input the dates you would like to retrieve data for, in our case this was the jupyter notebook for December 8-14.****

In [10]:
dates = ['2015-12-08','2015-12-09','2015-12-10','2015-12-11','2015-12-12','2015-12-13','2015-12-14']
datamap = {}

****Run entire program on the dates listed above in 5 minute windows. Print the information for each window, so you can easily check progress of the program.****

In [11]:
%%time
import pymysql
import pandas as a
import numpy as np
import math
import matplotlib.pyplot as plt
import networkx as nx
import haversine as hs
from haversine import Unit
import datetime
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon, shape
import gmplot

conn=pymysql.connect(host='127.0.0.1',port=int(3306),user='root',passwd=' ',db=' ')

merged_dist = 0.0
total_dist = 0.0
total_rides = 0
merge_rides = 0

for date in dates:  
    curr_data = []
    for time in times:

        df=a.read_sql_query(
            """SELECT Lpep_pickup_datetime, lpep_dropoff_datetime ,Pickup_longitude, Pickup_latitude, Dropoff_longitude, Dropoff_latitude, Trip_distance, Passenger_count FROM green_dec_data 
            WHERE Lpep_pickup_datetime >= '""" + date + """ """ + time[0] + """' AND Lpep_pickup_datetime <= '""" + date + """ """ + time[1] + """'
            UNION
            SELECT tpep_pickup_datetime, tpep_dropoff_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, trip_distance, passenger_count FROM yellow_dec_data
            WHERE tpep_pickup_datetime >= '""" + date + """ """ + time[0] + """' AND tpep_pickup_datetime <= '""" + date + """ """ + time[1] + """'
            ORDER BY Lpep_pickup_datetime;""",
        conn)

        df = df[(df != 0).all(1)]
        df['points'] = 1

        street_map = gpd.read_file('C:/Users/Michael/Desktop/School/CS-480/Project/geo/manhattan.geojson')
        geometry = [Point(xy) for xy in zip(df['Pickup_latitude'], df['Pickup_longitude'])]
        df['Points'] = geometry


        l = []
        with open('C:/Users/Michael/Desktop/School/CS-480/Project/geo/manhattan.geojson') as f:
            js = json.load(f)
        coords = js['features'][0]['geometry']['coordinates'][0][0]

        def coord(geometry):
            coord_list = []
            poly = Polygon(coord_list)
            res = []
            for i in range(0, len(geometry)):
                res.append(False)

            for i in js:
                for j in js[i]:
                    if type(j) is dict:
                        coord = j['geometry']['coordinates'][0][0]
                        coord_list = []
                        for i in coord:
                            pair = (i[1],i[0])
                            coord_list.append(pair)
                        poly = Polygon(coord_list)
                        for i in range(0, len(geometry)):
                            if res[i]:
                                continue
                            res[i] = poly.contains(geometry[i])
            return res

        df['Contains_pickup'] = coord(geometry)
        df['Contains_dropoff'] = coord([Point(xy) for xy in zip(df['Dropoff_latitude'], df['Dropoff_longitude'])])
        
        df = df[df['Contains_pickup'] == True]
        df = df[df['Contains_dropoff'] == True]

        pickup_lon = df['Pickup_longitude'].tolist()
        pickup_lat = df['Pickup_latitude'].tolist()
        dropoff_lon = df['Dropoff_longitude'].tolist()
        dropoff_lat = df['Dropoff_latitude'].tolist()
        before_distance = []

        for i in range(0, len(dropoff_lat)):
            before_distance.append(hs.haversine((pickup_lat[i], pickup_lon[i]), (dropoff_lat[i], dropoff_lon[i]), unit = Unit.MILES))

        df['distance'] = before_distance
        df = df[df['distance'] != 0.0]
        df = df[df['distance'] <= 22.82]

        pickup_longitude = df['Pickup_longitude'].tolist()
        pickup_latitude = df['Pickup_latitude'].tolist()
        dropoff_longitude = df['Dropoff_longitude'].tolist()
        dropoff_latitude = df['Dropoff_latitude'].tolist()
        pickup_time= df['Lpep_pickup_datetime'].tolist()
        dropoff_time = df['lpep_dropoff_datetime'].tolist()
        passengers = df['Passenger_count'].tolist()
        total_distance = df['distance'].tolist()
        total_time = []
        total_rides += len(passengers)

        for i in range(0, len(dropoff_time)):
            total_time.append(int((dropoff_time[i] - pickup_time[i]).total_seconds()))

        #pandas datetime object was extremely slow when doing calculations
        pickup_time = [x.to_pydatetime() for x in pickup_time]
        dropoff_time = [x.to_pydatetime() for x in dropoff_time]


        def sequence(p1, p2, d1, d2, a,b,c,d,taxi):
            trip1 = hs.haversine(p1, p2, unit = Unit.MILES)
            trip2 = hs.haversine(p2, d1, unit = Unit.MILES)
            trip3 = hs.haversine(d1, d2, unit = Unit.MILES)

            time1 = pickup_time[b] + datetime.timedelta(minutes=trip1/taxi)
            time2 = dropoff_time[c] + datetime.timedelta(minutes=trip2/taxi)
            time3 = dropoff_time[d] + datetime.timedelta(minutes=trip3/taxi) 

            res = trip1 + trip2 + trip3

            if abs(time1 - pickup_time[b]).total_seconds() >= 300.0:
                return (res, False)
            elif abs(time2 - dropoff_time[c]).total_seconds() >= 300.0:
                return (res, False)
            elif abs(time3 - dropoff_time[d]).total_seconds() >= 300.0:
                return (res, False)

            return (trip1+trip2+trip3, True)

        trips = {}
        G = nx.Graph()
        for i in range(0, len(pickup_longitude)):
            
            if passengers[i] >= 3:
                continue
                
            t1_pickup = (pickup_longitude[i], pickup_latitude[i])
            t1_dropoff = (dropoff_longitude[i], dropoff_latitude[i])

            d1 = hs.haversine(t1_pickup, t1_dropoff, unit = Unit.MILES)

            if d1 == 0.0:
                continue

            if total_time[i] == 0.0:
                continue

            s1 = d1/(total_time[i]/60)

            for j in range(i, len(pickup_longitude)):

                if passengers[i] + passengers[j] > 3:
                    continue

                if i == j:
                    continue

                t2_pickup = (pickup_longitude[j], pickup_latitude[j])
                t2_dropoff = (dropoff_longitude[j], dropoff_latitude[j])
                d2 = hs.haversine(t2_pickup, t2_dropoff, unit = Unit.MILES)

                if d2 == 0.0:
                    continue

                if total_time[j] == 0.0:
                    continue

                s2 = d2/(total_time[j]/60)

                taxi = (s1+s2)/ 2.0

                seq1 = sequence(t1_pickup, t2_pickup, t1_dropoff, t2_dropoff,i,j,i,j,taxi)
                seq2 = sequence(t1_pickup, t2_pickup, t2_dropoff, t1_dropoff,i,j,j,i,taxi)
                seq3 = sequence(t2_pickup, t1_pickup, t1_dropoff, t2_dropoff,j,i,i,j,taxi)
                seq4 = sequence(t2_pickup, t1_pickup, t2_dropoff, t1_dropoff,j,i,j,i,taxi)

                curr_distance = d1 + d2
                curr_min = 1000000
                merged_trip = ()
                flag = 0

                if seq1[0] < curr_distance and seq1[1]:
                    if seq1[0] < curr_min:
                        curr_min = seq1[0]
                        merged_trip = (t1_pickup, t1_dropoff, t2_pickup, t2_dropoff)

                if seq2[0] < curr_distance and seq2[1]: 
                    if seq2[0] < curr_min:
                        curr_min = seq2[0]
                        merged_trip = (t1_pickup, t1_dropoff, t2_pickup, t2_dropoff)

                if seq3[0] < curr_distance and seq3[1]:
                    if seq3[0] < curr_min:
                        curr_min = seq3[0]
                        merged_trip = (t1_pickup, t1_dropoff, t2_pickup, t2_dropoff)

                if seq4[0] < curr_distance and seq4[1]:
                    if seq4[0] < curr_min:
                        curr_min = seq4[0]
                        merged_trip = (t1_pickup, t1_dropoff, t2_pickup, t2_dropoff)

                if curr_min != 1000000:
                    if i not in trips:
                        trips[i] = [(j,curr_min)]
                        G.add_edge(i, j, weight=curr_min)
                    else:
                        trips[i].append((j,curr_min))
                        G.add_edge(i, j, weight=curr_min)

        # https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.matching.max_weight_matching.html
        A = nx.max_weight_matching(G, maxcardinality=True)

        merged_distance = 0.0
        already_matched = set()

        for i in range(0, len(total_distance)):
            already_matched.add(i)

        for i in A:
            merge_rides += 1
            already_matched.remove(i[0])
            already_matched.remove(i[1])
            if i[0] < i[1]:
                trip = trips[i[0]]
                search = i[1]
            else:
                trip = trips[i[1]]
                search = i[0]
            for j in trip:
                if j[0] == search:
                    merged_distance += j[1]

        for i in already_matched:
            merge_rides += 1
            merged_distance += total_distance[i]

        merged_dist += merged_distance
        total_dist += sum(total_distance)
        print(date)
        print(time)
        print(merged_distance)
        print(sum(total_distance))
        if sum(total_distance) == 0.0:
            print('0.0')
        else:
            print(1 - (merged_distance/sum(total_distance)))
        print(merged_dist)
        print(total_dist)
        print(total_rides)
        print(merge_rides)
        print()
        res = [time, merged_distance, sum(total_distance)]
        curr_data.append(res)
    datamap[date] = curr_data

2015-12-08
('00:00:00', '00:05:00')
1173.7779852470503
1434.8161276088115
0.1819314247580932
1173.7779852470503
1434.8161276088115
792
551

2015-12-08
('00:05:01', '00:10:00')
1004.4449917028985
1234.197459608457
0.18615535635476532
2178.222976949949
2669.0135872172687
1487
1028

2015-12-08
('00:10:01', '00:15:00')
934.492946903323
1164.5313163871347
0.19753729783539642
3112.715923853272
3833.5449036044033
2178
1505

2015-12-08
('00:15:01', '00:20:00')
891.8045962561536
1121.5715466407294
0.20486160786867447
4004.520520109426
4955.116450245133
2828
1943

2015-12-08
('00:20:01', '00:25:00')
817.4068329321782
1032.5159637172792
0.2083349200826512
4821.927353041604
5987.632413962412
3439
2363

2015-12-08
('00:25:01', '00:30:00')
745.2828840274994
902.506194615344
0.17420745865888987
5567.210237069104
6890.138608577756
3985
2735

2015-12-08
('00:30:01', '00:35:00')
724.7817938704416
943.2605393343558
0.2316207838166232
6291.992030939546
7833.399147912111
4540
3103

2015-12-08
('00:35:01', 

2015-12-08
('04:50:01', '04:55:00')
192.8253379063435
253.3338423621965
0.23884887976926028
22340.260178599838
27341.20758228766
15665
10947

2015-12-08
('04:55:01', '05:00:00')
131.84306605113392
169.05027491390453
0.2200955241375374
22472.10324465097
27510.257857201566
15771
11017

2015-12-08
('05:00:01', '05:05:00')
145.97739719676412
181.5904626752302
0.19611748851677924
22618.080641847733
27691.848319876797
15879
11095

2015-12-08
('05:05:01', '05:10:00')
186.9819645742961
234.4663790916144
0.20252120880309443
22805.06260642203
27926.31469896841
16005
11183

2015-12-08
('05:10:01', '05:15:00')
210.77055961004052
257.1877221735501
0.18047969853003842
23015.833166032073
28183.50242114196
16147
11279

2015-12-08
('05:15:01', '05:20:00')
219.50929137274673
260.35262809171513
0.1568769903278272
23235.34245740482
28443.855049233673
16296
11388

2015-12-08
('05:20:01', '05:25:00')
223.78285917914135
293.3310782304936
0.23709802408561242
23459.12531658396
28737.186127464167
16485
11506

2

2015-12-08
('09:40:01', '09:45:00')
1981.4604369742249
2122.0337338419786
0.06624460988810166
102186.17884649601
119033.67009430112
79053
58470

2015-12-08
('09:45:01', '09:50:00')
1839.3452386026156
1982.6592473013561
0.07228373150545586
104025.52408509863
121016.32934160248
80541
59703

2015-12-08
('09:50:01', '09:55:00')
1883.4870014030332
2053.1940326562317
0.08265513563452509
105909.01108650166
123069.52337425872
82090
60964

2015-12-08
('09:55:01', '10:00:00')
1718.9444977470043
1852.9253186616552
0.07230772852271428
107627.95558424866
124922.44869292037
83519
62134

2015-12-08
('10:00:01', '10:05:00')
1707.9163022007308
1849.6623525921868
0.07663347323515979
109335.87188644939
126772.11104551256
84924
63288

2015-12-08
('10:05:01', '10:10:00')
1694.3318119186008
1829.1982321421278
0.0737298002226846
111030.20369836799
128601.30927765468
86282
64414

2015-12-08
('10:10:01', '10:15:00')
1690.4911642461798
1827.17041588034
0.07480377880806888
112720.69486261417
130428.47969353502
8

2015-12-08
('14:25:01', '14:30:00')
1727.5963861771443
1847.4781794664693
0.06488942311835344
198646.1090278726
222400.83219809254
156857
124136

2015-12-08
('14:30:01', '14:35:00')
1730.6722336351459
1846.1194986050289
0.06253509865266993
200376.78126150774
224246.95169669756
158235
125297

2015-12-08
('14:35:01', '14:40:00')
1725.472032515331
1845.1706257029273
0.06487128698030109
202102.25329402307
226092.12232240048
159656
126513

2015-12-08
('14:40:01', '14:45:00')
1791.5892599431727
1916.4829227963328
0.0651681584884295
203893.84255396624
228008.6052451968
161108
127749

2015-12-08
('14:45:01', '14:50:00')
1738.4498762665112
1872.7224703855695
0.07169914188695203
205632.29243023274
229881.3277155824
162534
128957

2015-12-08
('14:50:01', '14:55:00')
1778.752710133883
1911.5163378352338
0.06945461311186263
207411.04514036662
231792.84405341762
164001
130182

2015-12-08
('14:55:01', '15:00:00')
1674.7956007817875
1828.1386212563118
0.08387931784360303
209085.84074114842
233620.9826

2015-12-08
('19:10:01', '19:15:00')
2340.194769562342
2558.423328106371
0.0852980646895336
296485.44125126296
328415.50304289226
236615
190236

2015-12-08
('19:15:01', '19:20:00')
2287.61047796278
2512.3748141286987
0.08946289976397015
298773.0517292257
330927.87785702094
238416
191683

2015-12-08
('19:20:01', '19:25:00')
2240.264739426362
2488.5806727367953
0.0997821513406475
301013.3164686521
333416.45852975774
240209
193106

2015-12-08
('19:25:01', '19:30:00')
2298.5301708204834
2561.3318591028687
0.10260352923358951
303311.8466394726
335977.7903888606
242026
194518

2015-12-08
('19:30:01', '19:35:00')
2293.556983060338
2533.4413391120315
0.09468715629933344
305605.4036225329
338511.2317279726
243840
195959

2015-12-08
('19:35:01', '19:40:00')
2362.759209095391
2619.3965737967724
0.09797575795458491
307968.1628316283
341130.6283017694
245679
197430

2015-12-08
('19:40:01', '19:45:00')
2453.5016059767217
2721.3241430966177
0.09841625732065074
310421.664437605
343851.952444866
247557


...

2015-12-14
('02:20:01', '02:25:00')
255.15980005868704
309.0806886992737
0.17445570238472607
2794439.0503325937
3132316.712243825
2089891
1657173

2015-12-14
('02:25:01', '02:30:00')
279.5663704947476
331.0035188080999
0.15539758761046008
2794718.6167030884
3132647.7157626334
2090066
1657296

2015-12-14
('02:30:01', '02:35:00')
265.0979152645857
314.364996485333
0.15671936052538815
2794983.714618353
3132962.080759119
2090225
1657408

2015-12-14
('02:35:01', '02:40:00')
310.57220159587683
363.790469212448
0.14628824040327593
2795294.2868199493
3133325.8712283312
2090393
1657537

2015-12-14
('02:40:01', '02:45:00')
251.21465041262312
288.2793198028551
0.1285720717517279
2795545.501470362
3133614.150548134
2090533
1657642

2015-12-14
('02:45:01', '02:50:00')
278.62823087579187
315.03023279918995
0.11555082063060862
2795824.1297012377
3133929.1807809332
2090682
1657757

2015-12-14
('02:50:01', '02:55:00')
236.13745999272868
265.0928872415618
0.10922747701807611
2796060.2671612306
3134194.2

2015-12-14
('07:00:01', '07:05:00')
1359.9878382999066
1696.8021951137557
0.19849948201609247
2815988.158279972
3159551.5207780674
2106301
1667880

2015-12-14
('07:05:01', '07:10:00')
1423.9109390116246
1790.0326224706687
0.20453352573748607
2817412.069218984
3161341.553400538
2107408
1668610

2015-12-14
('07:10:01', '07:15:00')
1551.4588086124777
1932.8880536372346
0.19733643875908802
2818963.5280275964
3163274.4414541754
2108606
1669408

2015-12-14
('07:15:01', '07:20:00')
1623.0056186246004
2012.944169533606
0.19371553211004033
2820586.533646221
3165287.385623709
2109914
1670289

2015-12-14
('07:20:01', '07:25:00')
1581.1408551232942
1913.4241602865443
0.17365898897894605
2822167.674501344
3167200.809783995
2111160
1671136

2015-12-14
('07:25:01', '07:30:00')
1594.4426663864765
1979.1763410540173
0.19439080120705643
2823762.1171677304
3169179.986125049
2112468
1672011

2015-12-14
('07:30:01', '07:35:00')
1819.6052857353213
2176.597507840654
0.1640138890260402
2825581.7224534657
3171

2015-12-14
('11:40:01', '11:45:00')
1508.0556958392513
1673.7120892935275
0.09897544178234363
2912876.230970114
3268074.0052686464
2183710
1728145

2015-12-14
('11:45:01', '11:50:00')
1677.8302268157347
1850.941734123467
0.09352617865613755
2914554.06119693
3269924.94700277
2185070
1729232

2015-12-14
('11:50:01', '11:55:00')
1642.2049821585151
1833.6795671340542
0.10442096231393572
2916196.2661790885
3271758.626569904
2186435
1730309

2015-12-14
('11:55:01', '12:00:00')
1568.628966976249
1731.7805847905727
0.09421032851806321
2917764.895146065
3273490.4071546947
2187747
1731372

2015-12-14
('12:00:01', '12:05:00')
1558.4391212513003
1743.9218825526375
0.10635955839365907
2919323.3342673164
3275234.329037247
2189085
1732430

2015-12-14
('12:05:01', '12:10:00')
1505.718266475943
1656.649442987227
0.09110628512881336
2920829.0525337923
3276890.9784802343
2190366
1733458

2015-12-14
('12:10:01', '12:15:00')
1518.045821626015
1677.2368665935142
0.09491267938250003
2922347.0983554185
327856

2015-12-14
('16:20:01', '16:25:00')
1394.1412815155618
1575.4323496010181
0.11507385139791548
3001139.073343116
3366340.1197613706
2257945
1786748

2015-12-14
('16:25:01', '16:30:00')
1343.800264766811
1523.7086413741872
0.11807268904448964
3002482.8736078828
3367863.8284027446
2259090
1787611

2015-12-14
('16:30:01', '16:35:00')
1384.244568927919
1568.3575225752327
0.11739220872610812
3003867.118176811
3369432.18592532
2260286
1788535

2015-12-14
('16:35:01', '16:40:00')
1437.434573329163
1614.433318046865
0.10963521548962729
3005304.55275014
3371046.6192433666
2261512
1789487

2015-12-14
('16:40:01', '16:45:00')
1445.1394174826548
1623.6052055742387
0.10991944807695031
3006749.6921676225
3372670.2244489407
2262719
1790422

2015-12-14
('16:45:01', '16:50:00')
1356.2842190590402
1554.7997355741709
0.12767915505325256
3008105.9763866817
3374225.024184515
2263910
1791320

2015-12-14
('16:50:01', '16:55:00')
1510.6901500797833
1728.7242959889772
0.1261243024206239
3009616.6665367614
33759

2015-12-14
('21:00:01', '21:05:00')
2221.9316944379166
2577.7200702053824
0.13802444256063773
3119561.786453203
3500894.2078954885
2356261
1861251

2015-12-14
('21:05:01', '21:10:00')
2429.9064967687304
2800.9135762430597
0.13245930992700283
3121991.6929499716
3503695.1214717315
2357983
1862510

2015-12-14
('21:10:01', '21:15:00')
2335.4972545505343
2734.8277772280603
0.14601669838320697
3124327.190204522
3506429.9492489593
2359721
1863775

2015-12-14
('21:15:01', '21:20:00')
2255.424229526589
2687.0606011362506
0.16063514586427263
3126582.6144340485
3509117.0098500955
2361379
1864958

2015-12-14
('21:20:01', '21:25:00')
2265.8317595735043
2629.252880185422
0.1382222012004749
3128848.446193622
3511746.262730281
2363003
1866149

2015-12-14
('21:25:01', '21:30:00')
2249.0191962837903
2658.8247475398393
0.15413033583173708
3131097.465389906
3514405.087477821
2364611
1867289

2015-12-14
('21:30:01', '21:35:00')
2233.5440491469335
2635.8998760523104
0.15264457901487916
3133331.009439053
351

****Loop through map that has all stored information from each day, create information variables, then display information.****

In [12]:
distance_total = 0.0
distance_after_merged = 0.0

for key in datamap:
    for vals in datamap[key]:
        distance_after_merged += vals[1]
        distance_total += vals[2]

print('Week of december 8 - december 14')
print('Distance Before        : '+ str(distance_total))
print('Distance After Merged  : '+ str(distance_after_merged))
print('Percent Miles Saved    : '+ str((1-(distance_after_merged/distance_total))))
print('Total Miles Saved      : '+ str((distance_total-distance_after_merged)))

Week of december 8 - december 14
Distance Before        : 3577104.548195456
Distance After Merged  : 3183064.815624031
Percent Miles Saved    : 0.11015605701829623
Total Miles Saved      : 394039.7325714254
